# Notebook Summary
This notebook serves the purpose of learning the basic DSPy syntax and learning how each component in DSPy works. This will follow a few experiments and tutorial from the DSPy tutorials page:
- https://dspy.ai/tutorials/classification_finetuning/

# Imports

In [25]:
import dspy
from dspy.datasets import DataLoader
from typing import Literal
from datasets import load_dataset
import random

# Load Dataset

The tutorial uses the BANKING77 dataset. It is a collection of customer service queries related to online banking, annotated with 77 different intent categories

In [6]:
# Load the Banking77 dataset.
CLASSES = load_dataset("PolyAI/banking77", split="train", trust_remote_code=True).features['label'].names
kwargs = dict(fields=("text", "label"), input_keys=("text",), split="train", trust_remote_code=True)

In [19]:
print(f"Number of Classes: {len(CLASSES)}") # actually has 77 classes as the name suggests
print(CLASSES)

Number of Classes: 77
['activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawal_not_recognised', 'change_pin', 'compromised_card', 'contactless_not_working', 'country_support', 'declined_card_payment', 'declined_cash_withdrawal', 'declined_transfer', 'direct_debit_payment_not_recognised', 'disposable_card_limits', 'edit_personal_details', 'exchange_charge', 'exchange_rate', 'exchange_via_app', 'extra_charge_on_statement', 'failed_transfer', 'fiat_currency_support', 'get_disposable_virtual_card', 'get_physical_card', 'getting_spare_card', 

The tutorial uses 1,000 data points and provides 'hints,' but I don't actually understand their purpose or why `DSPy.Example` is being used. 

Let's take a closer look.

In [14]:
x = DataLoader().from_huggingface(dataset_name="PolyAI/banking77", **kwargs)[:1]
print(f"Raw Data : \n {x[0]}")

Raw Data : 
 Example({'text': 'I am still waiting on my card?', 'label': 11}) (input_keys={'text'})


In [21]:
check = [
    dspy.Example(x, label=CLASSES[x.label]).with_inputs("text") # what is example for?
    for x in DataLoader().from_huggingface(dataset_name="PolyAI/banking77", **kwargs)[:1]
]
print(f"DSPy Example Data : \n {check[0]}")

DSPy Example Data : 
 Example({'text': 'I am still waiting on my card?', 'label': 'card_arrival'}) (input_keys={'text'})


So it looks the same as the raw data, so it maintains the same format as the original although it converts the label numeric format to the corresponding text

Looking at the official DSPy documentation here [https://dspy.ai/deep-dive/data-handling/examples/]. We can see that it is similar to `Training` and `Testing` on traditional ML.

In [24]:
# move forward with the tutorial

# note to self: the 1000 rows of data will serve as our labelled training data that will be used for 'tuning' the model
raw_data = [
    dspy.Example(x, label=CLASSES[x.label]).with_inputs("text")
    for x in DataLoader().from_huggingface(dataset_name="PolyAI/banking77", **kwargs)[:1000]
] 
# while this will serve as our unlabeled training data
unlabelled_data = [
    dspy.Example(text = x.text).with_inputs("text")
    for x in DataLoader().from_huggingface(dataset_name="PolyAI/banking77", **kwargs)[:500]
]
random.Random(0).shuffle(raw_data)

# Load Model

In [29]:
# We will try a small gemma 3 model for this experiment
# Note: Make sure to have the ollama server running with the gemma3 model loaded
lm = dspy.LM('ollama_chat/gemma3:4b-it-qat', api_base='http://localhost:11434', api_key='')
dspy.configure(lm=lm)

# Creating a DSPy Signature

In [27]:
classify = dspy.ChainOfThought(f"text -> label : Literal{CLASSES}")
print(classify)

predict = Predict(StringSignature(text -> reasoning, label
    instructions='Given the fields `text`, produce the fields `label`.'
    text = Field(annotation=str required=True json_schema_extra={'__dspy_field_type': 'input', 'prefix': 'Text:', 'desc': '${text}'})
    reasoning = Field(annotation=str required=True json_schema_extra={'prefix': "Reasoning: Let's think step by step in order to", 'desc': '${reasoning}', '__dspy_field_type': 'output'})
    label = Field(annotation=Literal['activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawa

The tutorial proceeds with using a teacher student method for fine-tuning the model. As currently I don't have a paid model to use and tune, I will skip this part. Instead I will use MIPROv2

# Initial Performance

In [35]:
# this will set the language model for the classify program that we have made
classify.set_lm(lm)
# check the model's prediction using 1 example of data
print(f"Question: {raw_data[0].text}")
print(f"Answer : {raw_data[0].label}")

classify(text = raw_data[0].text) # this will return the model's prediction for the given example

Question: What if there is an error on the exchange rate?
Answer : card_payment_wrong_exchange_rate


Prediction(
    reasoning='The user is reporting an error related to the exchange rate. This falls under the category of incorrect exchange rates, specifically impacting a transaction or payment.',
    label='card_payment_wrong_exchange_rate'
)

Ok, looking at the example it's working quite well. The model is able to select the correct labels from the label list even with a small 4B "QUANTIZED" model

In [ ]:
# Evaluate on the dataset
# Since I don't want to break my PC let's use a small devset of 100 examples
devset = raw_data[500:600]
metric = (lambda x, y, trace=None: x.label == y.label) # check the accuracy by comparing the label of the example with the model's prediction

# so evaluate takes 2 main arguments, the devset and the metric
evaluate = dspy.Evaluate(devset=devset, metric=metric, display_progress=True, display_table=5, num_threads=16)

In [37]:
evaluate(classify)

  0%|          | 0/100 [00:00<?, ?it/s]

2025/06/05 23:55:29 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 1.00 / 1 (100.0%):   1%|          | 1/100 [00:18<30:46, 18.65s/it]

2025/06/05 23:55:30 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/06/05 23:55:30 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 9.00 / 16 (56.2%):  15%|█▌        | 15/100 [00:33<01:33,  1.09s/it]

...

2025/06/05 23:57:36 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/06/05 23:57:43 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.
2025/06/05 23:57:46 ERROR dspy.utils.parallelizer: Error for Example({'text': 'Please check my payment from last Saturday as I feel I have been overcharged on the exchange rate.  Thank you.', 'label': 'card_payment_wrong_exchange_rate'}) (input_keys={'text'}): Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: 'wrong_exchange_rate' is not one of ('activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'ca

Average Metric: 61.00 / 100 (61.0%): : 101it [02:37,  1.56s/it]                       

2025/06/05 23:57:49 INFO dspy.evaluate.evaluate: Average Metric: 61 / 100 (61.0%)


,text,example_label,reasoning,pred_label,<lambda>
0,Which fiat currencies do you currently support? Will this change i...,fiat_currency_support,The user is asking about the fiat currencies supported by the serv...,fiat_currency_support,✔️ [True]
1,I didn't receive my money earlier and it says the transaction is s...,pending_cash_withdrawal,The user is reporting that they haven't received their money and t...,failed_transfer,
2,what currencies do you accept?,fiat_currency_support,"The input text is a list of strings, each representing a potential...",Refund_not_showing_up,
3,Where can I find your exchange rates?,exchange_rate,The user is asking to find the exchange rates. This falls under th...,exchange_rate,✔️ [True]
4,why hasnt my card come in yet?,card_arrival,The user is asking about the delivery status of their card. This d...,card_arrival,✔️ [True]


61.0

So without using any prompt tuning, it is able to get an accuracy of 61%

# MIPROv2

In [40]:
optimizer = dspy.MIPROv2(metric=metric, auto="light", num_threads=4) # can choose between light, medium, and heavy optimization levels
optimized_program = optimizer.compile(classify, 
                                      trainset=devset,
                                      requires_permission_to_run=False
                                      )

2025/06/06 00:04:19 INFO dspy.teleprompt.mipro_optimizer_v2: 
RUNNING WITH THE FOLLOWING LIGHT AUTO RUN SETTINGS:
num_trials: 10
minibatch: True
num_fewshot_candidates: 6
num_instruct_candidates: 3
valset size: 80

2025/06/06 00:04:19 INFO dspy.teleprompt.mipro_optimizer_v2: 
==> STEP 1: BOOTSTRAP FEWSHOT EXAMPLES <==
2025/06/06 00:04:19 INFO dspy.teleprompt.mipro_optimizer_v2: These will be used as few-shot example candidates for our program and for creating instructions.

2025/06/06 00:04:19 INFO dspy.teleprompt.mipro_optimizer_v2: Bootstrapping N=6 sets of demonstrations...


Bootstrapping set 1/6
Bootstrapping set 2/6
Bootstrapping set 3/6


 35%|███▌      | 7/20 [00:12<00:23,  1.79s/it]


Bootstrapped 4 full traces after 7 examples for up to 1 rounds, amounting to 7 attempts.
Bootstrapping set 4/6


 15%|█▌        | 3/20 [00:02<00:13,  1.23it/s]


...

2025/06/06 00:18:07 ERROR dspy.utils.parallelizer: Error for Example({'text': 'What is this extra fee on my statement.', 'label': 'extra_charge_on_statement'}) (input_keys={'text'}): Both structured output format and JSON mode failed. Please choose a model that supports `response_format` argument. Original error: 'transaction_status' is not one of ('activate_my_card', 'age_limit', 'apple_pay_or_google_pay', 'atm_support', 'automatic_top_up', 'balance_not_updated_after_bank_transfer', 'balance_not_updated_after_cheque_or_cash_deposit', 'beneficiary_not_allowed', 'cancel_transfer', 'card_about_to_expire', 'card_acceptance', 'card_arrival', 'card_delivery_estimate', 'card_linking', 'card_not_working', 'card_payment_fee_charged', 'card_payment_not_recognised', 'card_payment_wrong_exchange_rate', 'card_swallowed', 'cash_withdrawal_charge', 'cash_withdrawal_not_recognised', 'change_pin', 'compromised_card', 'contactless_not_working', 'country_support', 'declined_card_payment', 'declined_cash

Average Metric: 49.00 / 64 (76.6%):  80%|████████  | 64/80 [00:14<00:04,  3.87it/s]

2025/06/06 00:18:11 WARNING dspy.adapters.json_adapter: Failed to use structured output format, falling back to JSON mode.


Average Metric: 59.00 / 79 (74.7%): 100%|██████████| 80/80 [00:17<00:00,  4.64it/s]

2025/06/06 00:18:13 INFO dspy.evaluate.evaluate: Average Metric: 59.0 / 80 (73.8%)
2025/06/06 00:18:13 INFO dspy.teleprompt.mipro_optimizer_v2: New best full eval score! Score: 73.75
2025/06/06 00:18:13 INFO dspy.teleprompt.mipro_optimizer_v2: Full eval scores so far: [62.5, 67.5, 73.75]
2025/06/06 00:18:13 INFO dspy.teleprompt.mipro_optimizer_v2: Best full score so far: 73.75
2025/06/06 00:18:13 INFO dspy.teleprompt.mipro_optimizer_v2: =======================
2025/06/06 00:18:13 INFO dspy.teleprompt.mipro_optimizer_v2: 

2025/06/06 00:18:13 INFO dspy.teleprompt.mipro_optimizer_v2: Returning best identified program with score 73.75!


The optimizer is able to raise the final score from 61% to 75% (which amounts to around 14% increase). I think that is quite good!

Looking at the log, we can see the mechanism behind MIPROv2. There are 3 main steps:
- Bootstrap Few-Shot Examples
    - this is used to find for good few shot examples in the given data (if the language model generate the correct answer it is kept)
- Propose Instruction Candidates
    - this is to change the format of the prompt itself, looking at the documentation there are a few things that it does
        - create a summary of the training data
        - summary of the LM's code
        - change instruction to be 'more creative' or some other thing and change the wording to make the instruction higher quality
- Find an Optimized Combination of Few-Shot Examples & Instructions
    - Use Bayesian Optimization to choose which combinations of instructions and examples

In [41]:
optimized_program.save(f"optimized.json")

Looking at the json file, it definitely overfit to the 100 samples that we selected as the prompt that is used in the final data was
- classify the inquiry into one of the following categories: `card_arrival`, `exchange_rate`, `transaction_status`, or `lost_found_card`

Which is only 4 out of the 77 groups that are in the dataset. Might need to do some kind of sampling to make sure it includes all sort of labels or just try again on binary classification dataset to see the actual efficacy of DSPy's prompt tuning